# Scraping Pune restaurants data from Zomato API

In [87]:
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pandas.io.json import json_normalize
from IPython.display import display
import plotly.graph_objects as go

- First one has to create an API key from the [Zomato developers page](https://developers.zomato.com/api) . Once you have the key, you can start using the Zomato API legally.

In [2]:
zomato_api_key = open("user_key").read()

base_url = 'https://developers.zomato.com/api/v2.1/'

In [3]:
def get_json_text(url, params, header):
    response = requests.get(url, params = params, headers=header)
    if response.status_code == 200:
        print('Status code: 200')
        return response.text
    else:
        print('Error! status code: {}'.format(response.status_code))

In [48]:
def resp_to_df(text, key, record_path = None, meta = None):
    '''Convert a json string to pandas dataframe'''
    js_str = json.loads(text) 
    df = json_normalize(js_str[key], record_path = record_path, meta = meta)
    return df

In [5]:
cuisine_api_url = base_url+'cuisines'
params = {'city_id':'5'}
header = {"User-agent": "curl/7.43.0", "Accept": "application/json", "user_key": zomato_api_key}
response_cuisine = get_json_text(cuisine_api_url, params, header)

Status code: 200


In [6]:
pune_cuisines = resp_to_df(response_cuisine, 'cuisines')
pune_cuisines

,cuisine.cuisine_id,cuisine.cuisine_name
0,1035,Afghan
1,152,African
2,1,American
3,2,Andhra
4,4,Arabian
...,...,...
94,95,Thai
95,93,Tibetan
96,142,Turkish
97,99,Vietnamese


#### What are the id's of my favourite cuisines? :)

In [7]:
pune_cuisines.loc[pune_cuisines['cuisine.cuisine_name'].isin(['Bengali', 'North Indian', 'Italian', 'Desserts']), :]

,cuisine.cuisine_id,cuisine.cuisine_name
11,10,Bengali
29,100,Desserts
47,55,Italian
72,50,North Indian


In [8]:
search_api_url = base_url+'search'
params = {'entity_id':'5', 'entity_type': 'city', 'order' : 'asc'}  #query parameters  'cuisines':1,2,3

response_search = get_json_text(search_api_url, params, header)
#pune_restaurants.columns = pune_restaurants.columns.str.replace(r'restaurant\.(R.)?(location.)?(user_rating.)?(rating_obj.)?', '')
#pune_restaurants.head()

Status code: 200


In [167]:
restaurant_data = resp_to_df(response_search, 'restaurants')
restaurant_data.head()

,restaurant.R.has_menu_status.delivery,restaurant.R.has_menu_status.takeaway,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,...,restaurant.has_table_booking,restaurant.events_url,restaurant.phone_numbers,restaurant.all_reviews.reviews,restaurant.establishment,restaurant.establishment_types,restaurant.medio_provider,restaurant.book_url,restaurant.order_url,restaurant.order_deeplink
0,-1,-1,18893197,16793d99389b264237a278059b8f4c4e,18893197,FC Road Social,https://www.zomato.com/pune/fc-road-social-shi...,"CTS 1183, Unit 101, 1st Floor & Mezzanine Floo...",Shivaji Nagar,Pune,...,0,https://www.zomato.com/pune/fc-road-social-shi...,"+91 9172378889, 020 29805112","[{'review': {'rating': 2, 'review_text': 'Visi...",[Bar],[],NaN,NaN,NaN,NaN
1,-1,-1,19034129,16793d99389b264237a278059b8f4c4e,19034129,Huber & Holly,https://www.zomato.com/pune/huber-holly-korega...,"Shop 2, Ground Floor, Ganga Commerce Apartment...",Koregaon Park,Pune,...,0,https://www.zomato.com/pune/huber-holly-korega...,080 46971586,"[{'review': {'rating': 5, 'review_text': 'Pune...",[Dessert Parlour],[],NaN,NaN,NaN,NaN
2,-1,-1,18678581,16793d99389b264237a278059b8f4c4e,18678581,Hippie@Heart,https://www.zomato.com/pune/hippie@heart-decca...,"Lane 10, Ground Floor, Bhandarkar Road, Beside...",Deccan Gymkhana,Pune,...,1,https://www.zomato.com/pune/hippie@heart-decca...,080 46971749,"[{'review': {'rating': 5, 'review_text': 'Hipp...",[Casual Dining],[],1.0,https://www.zomato.com/pune/hippie@heart-decca...,NaN,NaN
3,1,-1,18650304,16793d99389b264237a278059b8f4c4e,18650304,Tatva,https://www.zomato.com/pune/tatva-wakad?utm_so...,"Soham Icon, Thergaon, Wakad Road, Wakad, Pune",Wakad,Pune,...,0,https://www.zomato.com/pune/tatva-wakad/events...,"+91 8888523700, +91 8888526700","[{'review': {'rating': 4, 'review_text': '', '...",[Casual Dining],[],NaN,NaN,https://www.zomato.com/pune/tatva-wakad/order?...,
4,-1,-1,18536004,16793d99389b264237a278059b8f4c4e,18536004,Culture - पुणे,https://www.zomato.com/pune/culture-fc-road?ut...,"Survey 1195/6A, Near Tukaram Paduka Mandir, Gh...",FC Road,Pune,...,0,https://www.zomato.com/pune/culture-fc-road/ev...,080 46971112,"[{'review': {'rating': 1, 'review_text': '#ank...",[Bar],[],1.0,NaN,NaN,NaN


In [168]:
restaurant_review_data = resp_to_df(response_search, key = 'restaurants', record_path = ['restaurant', 'all_reviews', 'reviews'], 
                             meta = [['restaurant', 'name']])
print('Shape of the review dataset: {}'.format(restaurant_review_data.shape))
display(restaurant_review_data.head())
print(restaurant_review_data.columns.values)

Shape of the review dataset: (100, 18)


,review.rating,review.review_text,review.id,review.rating_color,review.review_time_friendly,review.rating_text,review.timestamp,review.likes,review.user.name,review.user.zomato_handle,review.user.foodie_level,review.user.foodie_level_num,review.user.foodie_color,review.user.profile_url,review.user.profile_image,review.user.profile_deeplink,review.comments_count,restaurant.name
0,2,Visited for the first time and the experience ...,45189598,FF7800,an hour ago,Blah!,1569039124,0,Coby,superfunkie,Super Foodie,8,f58552,https://www.zomato.com/superfunkie?utm_source=...,https://b.zmtcdn.com/data/user_profile_picture...,zomato://u/304797,0,FC Road Social
1,4,"When you go.. ask for manu , he’ll get you the...",45172098,5BA829,24 hours ago,Great!,1568956227,0,Madhurima Patil,NaN,Foodie,2,ffd35d,https://www.zomato.com/users/madhurima-patil-3...,https://b.zmtcdn.com/data/user_profile_picture...,zomato://u/34544177,0,FC Road Social
2,5,Manu our host for the evening was fabulous! He...,45168929,305D02,yesterday,Insane!,1568919931,0,Aditi Madkar,NaN,Foodie,3,ffd35d,https://www.zomato.com/users/aditi-madkar-1443...,https://b.zmtcdn.com/data/user_profile_picture...,zomato://u/14435971,0,FC Road Social
3,4,Excellent place good food very courteous staff...,45167951,5BA829,yesterday,Great!,1568916447,0,Abdul Shaikh,NaN,Foodie,2,ffd35d,https://www.zomato.com/users/abdul-shaikh-3199...,https://b.zmtcdn.com/data/user_profile_picture...,zomato://u/31998707,0,FC Road Social
4,1,We ordered Budweiser tower but they gave us th...,45167288,CB202D,yesterday,Avoid!,1568914603,0,Nilesh Jagadale,NaN,Foodie,2,ffd35d,https://www.zomato.com/users/nilesh-jagadale-4...,https://b.zmtcdn.com/data/user_profile_picture...,zomato://u/44285648,0,FC Road Social


['review.rating' 'review.review_text' 'review.id' 'review.rating_color'
 'review.review_time_friendly' 'review.rating_text' 'review.timestamp'
 'review.likes' 'review.user.name' 'review.user.zomato_handle'
 'review.user.foodie_level' 'review.user.foodie_level_num'
 'review.user.foodie_color' 'review.user.profile_url'
 'review.user.profile_image' 'review.user.profile_deeplink'
 'review.comments_count' 'restaurant.name']


### Which are the top restaurants by average ratings?

In [169]:
pd.DataFrame(restaurant_review_data.groupby(['restaurant.name'])['review.rating'].mean()).sort_values('review.rating', ascending=False)

,review.rating
restaurant.name,
The Barkhana,5.0
Darshan,4.8
Pop Tate's,4.8
McDonald's,4.8
Hippie@Heart,4.8
Huber & Holly,4.6
Le Plaisir,4.6
Tatva,4.4
AB's - Absolute Barbecues,4.2


In [170]:
locationdetails_api_url = base_url+'location_details'
params = {'entity_id':'5', 'entity_type': 'city', 'order' : 'asc'}
response_loc = requests.get(locationdetails_api_url, params = params, headers=header)

In [171]:
js_str = json.loads(response_loc.text) 
df = json_normalize(js_str['best_rated_restaurant']).iloc[0]
df.head()
#response_loc.json()

restaurant.R.has_menu_status.delivery                                   1
restaurant.R.has_menu_status.takeaway                                  -1
restaurant.R.res_id                                              18893197
restaurant.apikey                        16793d99389b264237a278059b8f4c4e
restaurant.id                                                    18893197
Name: 0, dtype: object

In [226]:
def scattermap_restaurants(data_df, city_name = 'Pune', lat = 18.55, lon = 73.8, zoom = 9, marker_size = 8):

    mapbox_access_token = open("mapbox_token").read()
    fig = go.Figure(data=go.Scattermapbox(
        lon = data_df['restaurant.location.longitude'],
        lat = data_df['restaurant.location.latitude'],
        text = data_df['restaurant.name'] + ', ' + data_df['restaurant.location.locality'] + '<br> Rating: ' +
    data_df['restaurant.user_rating.aggregate_rating'].astype(str)+ ', '
    + '<br>' + data_df['restaurant.cuisines'] + '<br> Cost for two: ' + data_df['restaurant.average_cost_for_two'].astype(str),
        mode = 'markers',
    marker=go.scattermapbox.Marker(
             size = marker_size,
           symbol = 'marker',  #can't use symbol with color
 #         color = 'red',   
             opacity = 1, #colorscale = 'viridis', colorbar_title = "Aggregate rating<br>of restaurants",cmin=2        
            )
        ))

    fig.update_layout(
     title = 'Restaurants in {}'.format(city_name),
    autosize=True,
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat = lat,
            lon = lon ),
        pitch=0,    #Sets the pitch angle of the map (in degrees, where "0" means perpendicular to the surface of the map) 
        zoom= zoom,
        style='light'
    ), 
    )
   
    fig.show()

In [229]:
scattermap_restaurants(restaurant_data, marker_size = 10, zoom = 10)